In [1]:
import pandas
import geopandas 
import feedparser
import requests
from   zipfile import ZipFile
import datetime 
from   local_module import NhcDownloaderBot

%matplotlib inline

In [5]:
url = 'https://www.nhc.noaa.gov/index-at.xml'
f = feedparser.parse(url)
f

{'bozo': False,
 'entries': [{'title': 'Atlantic Tropical Weather Outlook',
   'title_detail': {'type': 'text/plain',
    'language': None,
    'base': 'https://www.nhc.noaa.gov/index-at.xml',
    'value': 'Atlantic Tropical Weather Outlook'},
   'summary': '<br />\n000<br />\nABNT20 KNHC 120005 CCA<br />\nTWOAT <br />\n<br />\nTropical Weather Outlook...Corrected<br />\nNWS National Hurricane Center Miami FL<br />\n800 PM EDT Sat Sep 11 2021<br />\n<br />\nCorrected to change the order of the system northeast of the Azores.<br />\n<br />\nFor the North Atlantic...Caribbean Sea and the Gulf of Mexico:<br />\n<br />\nA tropical wave and an upper-level trough continue to produce a <br />\nlarge area of disorganized showers and thunderstorms over portions <br />\nof southeastern Mexico and the southern and central Gulf of Mexico. <br />\nUpper-level winds are gradually becoming more conducive for <br />\ndevelopment, and a tropical depression is likely to form on Sunday <br />\nor Monday 

In [3]:
#f
#f.keys()
# len(f.entries)
# f['entries'][1]['nhc_atcf']
# f.status
# f.feed
# f.feed['subtitle']
# f.entries
# type(f.entries)
# for entry in f.entries:
#     print(entry.title)

In [4]:
df = pandas.DataFrame(f.entries)
df = df.drop(columns=['title_detail', 'summary', 'summary_detail', 'published_parsed', 'links', 'link', 'id', 
                 'guidislink', 'authors', 'author', 'author_detail'])
df.head()

,title,published
0,Atlantic Tropical Weather Outlook,"Sun, 12 Sep 2021 00:06:19 GMT"
1,There are no tropical cyclones at this time.,"Sun, 12 Sep 2021 00:06:19 GMT"


In [6]:
tc_list = [nhc_atcf[2:4] for nhc_atcf in df['nhc_atcf'] if pandas.isnull(nhc_atcf) == False] 
tc_list

KeyError: 'nhc_atcf'

In [5]:
#{key: value for (key, value) in iterable}
d = {nhc_name: [nhc_atcf    , nhc_type, nhc_center, nhc_movement, 
                nhc_pressure, nhc_wind, published , nhc_datetime]         
     for (nhc_name    , nhc_atcf, nhc_type  , nhc_center  , nhc_movement,
          nhc_pressure, nhc_wind, published , nhc_datetime) 
     in  zip(df['nhc_name']    , df['nhc_atcf']    , df['nhc_type'], df['nhc_center'], 
             df['nhc_movement'], df['nhc_pressure'], df['nhc_wind'], df['published'],
             df['nhc_datetime']) 
     if pandas.isnull(nhc_name) == False}   
d

{'Larry': ['AL122021',
  'Hurricane',
  '43.5, -58.2',
  'NNE at 35 mph',
  '967 mb',
  '80 mph',
  'Fri, 10 Sep 2021 20:43:39 GMT',
  '5:00 PM AST Fri Sep 10']}

In [12]:
class NhcDownloaderBot:
    def __init__(self, storm_number, year):
        self.storm_number = storm_number
        self.year         = year
        
        
        self.file_names     = file_names = [
                            f'al{self.storm_number}{self.year}_5day_latest.zip',
                            f'al{self.storm_number}{self.year}_fcst_latest.zip',
                            f'al{self.storm_number}{self.year}_best_track.zip',
                            f'gtwo_shapefiles.zip',
                            f'wsp_120hrhalfDeg_latest.zip',
                            f'wsp_120hr5km_latest.zip'
                            ]
        self.urls           = urls = [
                            f'https://www.nhc.noaa.gov/gis/forecast/archive',
                            f'https://www.nhc.noaa.gov/gis/forecast/archive',
                            f'https://www.nhc.noaa.gov/gis/best_track',
                            f'https://www.nhc.noaa.gov/xgtwo',
                            f'https://www.nhc.noaa.gov/gis/forecast/archive',
                            f'https://www.nhc.noaa.gov/gis/forecast/archive'
                            ]
        self.gdf_names      = gdf_names = {
                            file_names[0] : ['track_line_gdf', 'cone_gdf', 'points_gdf'],
                            file_names[1] : ['init_radii_gdf', 'fcst_radii_gdf'],
                            file_names[2] : ['best_track_points_gdf', 'best_track_line_gdf', 'best_track_radii_gdf', 'best_track_swath_gdf'],
                            file_names[3] : ['gtwo_areas_gdf', 'gtwo_lines_gdf', 'gtwo_points_gdf'],
                            file_names[4] : ['wsp_34_gdf_points', 'wsp_50_gdf_points', 'wsp_64_gdf_points'],
                            file_names[5] : ['wsp_34_gdf_polygons', 'wsp_50_gdf_polygons', 'wsp_64_gdf_polygons']
                                          }
    def nhc_gis_downloader(self, file_names, urls, gdf_names):

        for idx, file_name in enumerate(file_names):
            url = f'{urls[idx]}/{file_name}'
            r = requests.get(url)
            with open(f'nhc_latest/{file_name}', 'wb') as code:
                code.write(r.content) 
        _  = [0, 3, 4, 5]  
        __ = [2, 7, 12]  
        for idx, file_name in enumerate([file_names[0],  file_names[3], 
                                        file_names[4],  file_names[5]]): 
            with ZipFile(f'nhc_latest/{file_name}', 'r') as zip_file_name:
                for idx2 in range(len(gdf_names[file_names[_[idx]]])):
                    gdf_names[file_names[_[idx]]][idx2]  = geopandas.read_file(
                    f'zip://./nhc_latest/{file_name}!{zip_file_name.namelist()[__[idx2]]}')
        _  = [0,5]                                 
        with ZipFile(f'nhc_latest/{file_names[1]}', 'r') as zip_file_name:
                for idx in range(2):
                    gdf_names[file_names[1]][idx]  = geopandas.read_file(
                    f'zip://./nhc_latest/{file_names[1]}!{zip_file_name.namelist()[_[idx]]}')
        _  = [2, 7, 13, 16]                                 
        with ZipFile(f'nhc_latest/{file_names[2]}', 'r') as zip_file_name:
            for idx in range(4):
                gdf_names[file_names[2]][idx]  = geopandas.read_file(
                f'zip://./nhc_latest/{file_names[2]}!{zip_file_name.namelist()[_[idx]]}')
        return gdf_names
    
    @classmethod
    def gdf_file(cls, storm_number, year):
        cls._                     = NhcDownloaderBot(storm_number, year)
        cls.track_line_gdf        = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[0]][0]
        cls.cone_gdf              = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[0]][1]
        cls.points_gdf            = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[0]][2]
        cls.init_radii_gdf        = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[1]][0]
        cls.fcst_radii_gdf        = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[1]][1]
        cls.best_track_points_gdf = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[2]][0]
        cls.best_track_line_gdf   = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[2]][1]
        cls.best_track_radii_gdf  = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[2]][2]
        cls.best_track_swath_gdf  = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[2]][3]
        cls.gtwo_areas_gdf        = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[3]][0]
        cls.gtwo_lines_gdf        = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[3]][1]
        cls.gtwo_points_gdf       = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[3]][2]
        cls.wsp_34_gdf_points     = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[4]][0]
        cls.wsp_50_gdf_points     = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[4]][1]
        cls.wsp_64_gdf_points     = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[4]][2]
        cls.wsp_34_gdf_polygons   = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[5]][0]
        cls.wsp_50_gdf_polygons   = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[5]][1]
        cls.wsp_64_gdf_polygons   = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[5]][2]

In [13]:
NhcDownloaderBot.gdf_file(12, 2021)
print('Done')

Done


In [11]:
NhcDownloaderBot.track_line_gdf

,STORMNAME,STORMTYPE,ADVDATE,ADVISNUM,STORMNUM,FCSTPRD,BASIN,geometry
0,Larry,PTC,1100 AM AST Sat Sep 11 2021,44,12.0,120.0,AL,"LINESTRING (-48.20000 54.00000, -44.90000 57.8..."


In [9]:
ct = []
for tc in tc_list:
    df = NhcDownloaderBot()
    df = df.gdf_file(tc, 2021)
    ct.append(NhcDownloaderBot.track_line_gdf)
    if tc == tc_list[-1]:
        print(f'--------------------done-----------------------')
    
#     NhcDownloaderBot.init_radii_gdf
    

--------------------done-----------------------


In [10]:
ct[0]

,STORMNAME,STORMTYPE,ADVDATE,ADVISNUM,STORMNUM,FCSTPRD,BASIN,geometry
0,Larry,HU,500 PM AST Fri Sep 10 2021,41,12.0,120.0,AL,"LINESTRING (-58.20000 43.50000, -53.30000 48.6..."
